In [1]:
# CONSTANTS
salary_cap = 94143000

In [2]:
import requests
from nba_py import player, team
from nba_py.constants import *
from bs4 import BeautifulSoup
from __future__ import print_function
import pandas as pd
import numpy as np
import re
import itertools
import datetime as dtime

In [3]:
def strip_name(name):
    return re.sub(r'[^\w\s]','',name.lower())

In [4]:
salaryDT = pd.read_csv('data/contracts_players_contracts.csv')
salaryDT['formatted_name'] = salaryDT.apply(lambda row: strip_name(row['Player']), axis=1)

In [5]:
def find_valid_team_players(team_name, valid_players):
    teamDT = team.TeamCommonRoster(team_id=TEAMS[team_name]['id'],season='2016-17').roster()
    roster = list(teamDT['PLAYER'])
    rookies = 0
    a = list(roster)
    
    for player in a:
        if(len(salaryDT[salaryDT['formatted_name'] == strip_name(player)]) != 0):
            if(salaryDT[salaryDT['formatted_name'] == strip_name(player)]['Signed Using'].iloc[0] == '1st Round Pick'):
                rookies = rookies + 1
                roster.remove(player)
        else:
            roster.remove(player)
        
    for _ in range(5-rookies):
        team_contracts = salaryDT[salaryDT.Player.isin(roster)][['Player','2016-17']]
        roster.remove(team_contracts.iloc[0]['Player'])
    
    valid_players.append(roster)
        
    return valid_players

In [6]:
def find_valid_players():
    valid_players = []
    for team_name in list(TEAMS.keys()):
        valid_players = find_valid_team_players(team_name, valid_players)
    valid_players = list(itertools.chain.from_iterable(valid_players))
    
    return valid_players

In [7]:
def create_master_player_DT():
    teamDTList = []
    for team_name in list(TEAMS.keys()):
        teamDT = team.TeamCommonRoster(team_id=TEAMS[team_name]['id'],season='2016-17').roster()
        teamDTList.append(teamDT)
    return pd.concat(teamDTList)

In [8]:
valid_players = find_valid_players()

In [9]:
masterPlayerDT = create_master_player_DT()
masterPlayerDT = masterPlayerDT[masterPlayerDT.PLAYER.isin(valid_players)]

masterPlayerIDs = list(masterPlayerDT['PLAYER_ID'])

In [51]:
list(team.TeamCommonRoster(team_id=TEAMS['NYK']['id'],season='2016-17').roster()['PLAYER'])

,TeamID,SEASON,LeagueID,PLAYER,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID
0,1610612752,2016,00,Mindaugas Kuzminskas,None,F,6-9,216,"OCT 19, 1989",26.0,R,None,1627851
1,1610612752,2016,00,Marshall Plumlee,None,C,7-0,250,"JUL 14, 1992",24.0,R,Duke,1627850
2,1610612752,2016,00,Ron Baker,None,G,6-4,212,"MAR 30, 1993",23.0,R,Wichita State,1627758
3,1610612752,2016,00,Maurice Ndour,None,F,6-9,200,"JUN 18, 1992",24.0,R,Ohio U.,1626254
4,1610612752,2016,00,Guillermo Hernangomez,None,C,6-10,240,"MAY 27, 1994",22.0,R,Spain,1626195
5,1610612752,2016,00,Chasson Randle,None,G,6-2,185,"FEB 05, 1993",23.0,R,Stanford,1626184
6,1610612752,2016,00,JP Tokoto,None,G,6-6,200,"SEP 15, 1993",22.0,R,North Carolina,1626160
7,1610612752,2016,00,Justin Holiday,None,G,6-6,185,"APR 05, 1989",27.0,3,Washington,203200
8,1610612752,2016,00,Brandon Jennings,None,G,6-1,169,"SEP 23, 1989",26.0,7,Oak Hill Academy (VA),201943
9,1610612752,2016,00,Courtney Lee,None,G,6-5,200,"OCT 03, 1985",30.0,8,Western Kentucky,201584


In [ ]:
statsDT = pd.read_csv('data/basketball research data set.csv')

In [3]:
# Only want to look at most recent season 
statsDT = statsDT[statsDT['season'] == 2015]

In [4]:
# Convert date string to datetime
statsDT['date'].apply(lambda x: dtime.datetime.strptime(x, '%Y-%m-%d').date())

0         2014-12-02
1         2014-12-04
2         2014-12-05
3         2014-12-08
4         2014-12-09
5         2014-12-11
6         2014-12-12
7         2014-12-15
8         2014-12-17
9         2014-12-19
10        2014-12-21
11        2014-12-23
12        2014-12-26
13        2014-12-28
14        2014-12-30
15        2014-12-31
16        2015-01-02
17        2015-01-04
18        2015-01-05
169       2014-11-07
170       2014-11-08
171       2014-11-10
172       2014-11-12
173       2014-11-14
174       2014-11-15
175       2014-11-19
176       2014-11-22
177       2014-11-24
178       2014-11-26
234       2015-03-21
             ...    
208478    2015-04-04
208479    2015-04-05
208480    2015-04-07
208481    2015-04-09
208482    2015-04-11
208483    2015-04-13
208484    2015-04-15
208485    2014-10-31
208486    2014-11-04
208487    2014-11-07
208488    2014-11-14
208489    2014-11-15
208490    2014-11-17
208491    2014-11-26
208492    2014-11-30
208493    2014-12-26
208494    201

In [ ]:
teamGameStatsDT = statsDT.groupby(['team', 'date'])
teamGameStatsDT.agg({
        'orb': np.sum,
        'drb': np.sum,
        'stl': np.sum,
        'blk': np.sum,
        'ast': np.sum,
        ''
    })

In [5]:
playerStatsDT = statsDT.groupby(['name', 'team'])
playerStatsDT.agg({
        'pts': np.mean,
        'ast': np.mean 
    })

ast        pts
name              team                     
A.J. Price        CLE   1.181818   2.000000
                  IND   2.700000  10.500000
                  PHO   1.200000   1.200000
Aaron Brooks      CHI   3.182927  11.634146
Aaron Gordon      ORL   0.702128   5.170213
Adreian Payne     ATL   0.000000   1.666667
                  MIN   1.034483   7.172414
Al Horford        ATL   3.184211  15.131579
Al Jefferson      CHO   1.738462  16.646154
Al-Farouq Aminu   DAL   0.797297   5.540541
Alan Anderson     BRK   1.121622   7.364865
Alec Burks        UTA   3.037037  13.851852
Alex Kirk         CLE   0.000000   0.400000
Alex Len          PHO   0.463768   6.260870
Alexey Shved      HOU   0.333333   2.888889
                  NYK   3.562500  14.750000
                  PHI   2.647059   9.764706
Alexis Ajinca     NOP   0.691176   6.485294
Allen Crabbe      POR   0.764706   3.372549
Alonzo Gee        DEN   0.487179   4.871795
                  POR   0.400000   3.400000
Amar'e Stoudemire DAL   0.347826  10.782609
                  NYK   1.027778  12.000000
Amir Johnson      TOR   1.560000   9.253333
Anderson Varejao  CLE   1.346154   9.807692
Andre Dawkins     BOS        NaN        NaN
                  MIA   0.250000   0.750000
Andre Drummond    DET   0.670732  13.780488
Andre Iguodala    GSW   2.961039   7.844156
Andre Miller      SAC   4.700000   5.733333
...                          ...        ...
Troy Daniels      MIN   0.684211   2.789474
Ty Lawson         DEN   9.600000  15.240000
Tyler Ennis       MIL   2.360000   3.960000
                  PHO   1.125000   3.500000
Tyler Hansbrough  TOR   0.283784   3.648649
Tyler Johnson     MIA   1.218750   6.187500
Tyler Zeller      BOS   1.378049  10.158537
Tyreke Evans      NOP   6.594937  16.620253
Tyrus Thomas      MEM   0.500000   2.000000
Tyson Chandler    DAL   1.173333  10.386667
Udonis Haslem     MIA   0.741935   4.209677
Vander Blue       LAL   4.000000  11.000000
Victor Claver     POR   0.300000   2.700000
Victor Oladipo    ORL   4.097222  17.944444
Vince Carter      MEM   1.212121   5.848485
Wayne Ellington   LAL   1.630769   9.969231
Wesley Johnson    LAL   1.631579   9.907895
Wesley Matthews   POR   2.316667  15.933333
Will Barton       DEN   1.928571  10.964286
                  POR   0.800000   2.800000
Will Bynum        WAS   2.571429   3.142857
Will Cherry       CLE   1.000000   1.875000
Willie Green      ORL   1.307692   5.884615
Wilson Chandler   DEN   1.717949  13.910256
Xavier Henry      LAL   0.333333   2.222222
Zach LaVine       MIN   3.584416  10.103896
Zach Randolph     MEM   2.154930  16.098592
Zaza Pachulia     MIL   2.438356   8.301370
Zoran Dragic      MIA   0.500000   2.200000
                  PHO   0.166667   1.666667

[590 rows x 2 columns]